In [2]:
import json
import sys
import requests
import pandas as pd

ötletek:

### 1. Bírókhoz kapcsolódó elemzés

Arra lennénk kíváncsiak, hogy tudunk-e torzítást kimutatni bírói teljesítményekben csapatok/meccsek tekintetében. Erre valószínűleg szofisztikáktabb ökonometriára lenne szükség de eléggé érdekesnek találtuk a kérdést, hogy gondolkodjunk róla. Van arról adat a bírói statisztikákról, és meccsenkénti fújásokról is, ebből össze lehet állítani csapat szintű szabálytalanság történelmet. Ebből meg lehetne valahogy nézni, hogy fúj-e haza valamelyik csapatnak a bíró. Nehéz kiszűrni az endogenitást, mert eltérőek a fújási szabályok ligánként, akár évenként is szabályváltoztatások miatt. Valószínűleg számítani fog a hazai/vendég csapat ellen mit fújt.
- ehhez mi kell(het?) : 
    - Bíró szintű összesítés a meccsekről, mit fújt a bíró ki ellen
    - Csapat szintű összessítés amibe : fújások/lapok/szabadrúgások/bíró

A bírók esetében arra van szükség, hogy megnézzük milyen csapatoknak átlagosan mennyit fújtak/adtak lapot/ítéltek szabadrúgást. Ennek vesszük az átlagát, és megnézzük, hogy a többi csapathoz képest az a csapat amelyiket vizsgáljuk túl/alúl/átlagosan van fújva. Erre mondjuk elég lehet egy egyszerű t-teszt is. Önmagában azonban ez nem zárja ki azt, hogy az adott csapat alapvetően aggresszívabb/békésebb. Tehát meg kell néznünk csapat szinten, hogy a konkrét csapat milyen átlagos bűntetéseket kapott az bíró álltal fújt meccseken és ennek a komplementer halmazában. Így megtudjuk mondani bíró-csapat párok esetében, hogy az adott csapat magához képest több bűntetést kap-e az adott bírónál és az adott bíró máshogy, fúj ha ez a csapat meccsét vezeti. Ebből a két adatból már össze lehet kovácsolni egy közepesen gyenge mondást arra, hogy korrupt/elfogult-e a bíró. Szofisztikált elemzés itt: https://rss.onlinelibrary.wiley.com/doi/pdf/10.1111/j.1467-985X.2006.00451.x 

### 2. Szabálytalanságok természete csapatonként, ligánként

A másik ötlet amivel szívesen dolgoznánk az szintén a szabálytalanságokhoz kapcsolódik. Az event adatban van megfigyelhető x és y koordináta. Amit szívesen csinálnák az csapat/liga szintű vizualizáció arról, hogy hol, milyen szabálytalanságok történnek. Ebből lehetne látni mintázatokat arról, hogy melyik csapat/liga eseténben hol történnek inkább szabálytalanságok, ebben van e különbség csapat/liga szinten. Ezen kívül, ha szétbontjuk a pályát különböző részekre(támadás/védekezés/középpálya/bal/jobb) akkor jó statisztikákat csinálhatunk a csapatok aggresszívabb oldaláról/stratégiáról.

### +1
A koordináták szerinti vizualizációt továbbgondolva, ezt nem csak szabálytalanságokra, hanem mondjuk góllövések helyére is megcsinálhatnánk játékosokra látva, hogy melyik játékos/csapat honnan lő gólt.

In [3]:
ws_pathlist = json.loads(
    requests.get(
        "https://borza-hotelcom-data.s3.eu-central-1.amazonaws.com/whoscored-matchlist.json"
    ).content
)

In [4]:
len(ws_pathlist)  # --> 40k

20

In [5]:
ws_url = ws_pathlist[4]

In [6]:
ws_dic = json.loads(requests.get(ws_url).content)

In [7]:
type(ws_dic)

dict

In [8]:
ws_dic.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [41]:
ws_dic["events"]

[{'id': 2189290775.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 168,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2189290785.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 163,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2189290839.0,
  'eventId': 3,
  'minute': 0,
  'second': 1,
  'teamId': 168,
  'playerId': 25028,
  'x': 49.8,
  'y': 49.9,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 1, 'displayName': 'Pass'},
  'outcomeTy

In [10]:
ws_dic['periodMinuteLimits']

{'1': 45, '2': 90, '3': 105, '4': 120}

In [11]:
ws_dic['timeStamp']

'2020-03-08 16:52:58'

In [12]:
ws_dic['attendance']

31379

In [13]:
ws_dic['venueName']

'Bramall Lane'

In [14]:
ws_dic['referee']

{'officialId': 113,
 'firstName': 'Simon',
 'lastName': 'Hooper',
 'hasParticipatedMatches': False,
 'name': 'Simon Hooper'}

In [15]:
ws_dic['weatherCode']

'5'

In [16]:
ws_dic['elapsed']

'FT'

In [17]:
ws_dic['startTime']

'2020-03-07T15:00:00'

In [18]:
ws_dic['startDate']

'2020-03-07T00:00:00'

In [19]:
ws_dic['score']

'1 : 0'

In [20]:
ws_dic['htScore']

'1 : 0'

In [21]:
ws_dic['ftScore']

'1 : 0'

In [22]:
ws_dic['etScore']

''

In [23]:
ws_dic['pkScore']

''

In [24]:
ws_dic['statusCode']

6

In [25]:
ws_dic['periodCode']

3

In [26]:
ws_dic['home'].keys()

dict_keys(['teamId', 'formations', 'stats', 'incidentEvents', 'shotZones', 'name', 'countryName', 'players', 'managerName', 'scores', 'field', 'averageAge'])

In [27]:
ws_dic['away'].keys()

dict_keys(['teamId', 'formations', 'stats', 'incidentEvents', 'shotZones', 'name', 'countryName', 'players', 'managerName', 'scores', 'field', 'averageAge'])

In [28]:
ws_dic['maxMinute']

94

In [29]:
ws_dic['minuteExpanded']

96

In [30]:
ws_dic['maxPeriod']

2

In [31]:
ws_dic['expandedMinutes'] ## gyakorlatilag tök fölösleges

{'1': {'0': 0,
  '1': 1,
  '2': 2,
  '3': 3,
  '4': 4,
  '5': 5,
  '6': 6,
  '7': 7,
  '8': 8,
  '9': 9,
  '10': 10,
  '11': 11,
  '12': 12,
  '13': 13,
  '14': 14,
  '15': 15,
  '16': 16,
  '17': 17,
  '18': 18,
  '19': 19,
  '20': 20,
  '21': 21,
  '22': 22,
  '23': 23,
  '24': 24,
  '25': 25,
  '26': 26,
  '27': 27,
  '28': 28,
  '29': 29,
  '30': 30,
  '31': 31,
  '32': 32,
  '33': 33,
  '34': 34,
  '35': 35,
  '36': 36,
  '37': 37,
  '38': 38,
  '39': 39,
  '40': 40,
  '41': 41,
  '42': 42,
  '43': 43,
  '44': 44,
  '45': 45,
  '46': 46},
 '2': {'45': 47,
  '46': 48,
  '47': 49,
  '48': 50,
  '49': 51,
  '50': 52,
  '51': 53,
  '52': 54,
  '53': 55,
  '54': 56,
  '55': 57,
  '56': 58,
  '57': 59,
  '58': 60,
  '59': 61,
  '60': 62,
  '61': 63,
  '62': 64,
  '63': 65,
  '64': 66,
  '65': 67,
  '66': 68,
  '67': 69,
  '68': 70,
  '69': 71,
  '70': 72,
  '71': 73,
  '72': 74,
  '73': 75,
  '74': 76,
  '75': 77,
  '76': 78,
  '77': 79,
  '78': 80,
  '79': 81,
  '80': 82,
  '81': 83,
 

In [32]:
ws_dic['expandedMaxMinute']

96

In [33]:
ws_dic['periodEndMinutes']

{'1': 46, '2': 94}

In [37]:
ws_dic['events']

[{'id': 2189290775.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 168,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2189290785.0,
  'eventId': 2,
  'minute': 0,
  'second': 0,
  'teamId': 163,
  'x': 0.0,
  'y': 0.0,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 32, 'displayName': 'Start'},
  'outcomeType': {'value': 1, 'displayName': 'Successful'},
  'qualifiers': [],
  'satisfiedEventsTypes': [],
  'isTouch': False},
 {'id': 2189290839.0,
  'eventId': 3,
  'minute': 0,
  'second': 1,
  'teamId': 168,
  'playerId': 25028,
  'x': 49.8,
  'y': 49.9,
  'expandedMinute': 0,
  'period': {'value': 1, 'displayName': 'FirstHalf'},
  'type': {'value': 1, 'displayName': 'Pass'},
  'outcomeTy

In [38]:
ws_dic['timeoutInSeconds']

0

In [39]:
def get_v_info(v):
    tname = type(v).__name__
    try:
        l = len(v)
    except TypeError:
        l = 0
    if isinstance(v, dict):
        keys = v.keys()
    else:
        keys = []
    return {"type": tname, "length": l, "keys": keys, "size": sys.getsizeof(v)}

In [40]:
df=pd.DataFrame({k: get_v_info(v) for k, v in ws_dic.items()}).T
df

,type,length,keys,size
playerIdNameDictionary,dict,36,"(296332, 302402, 42266, 18501, 108632, 298805,...",1192
periodMinuteLimits,dict,4,"(1, 2, 3, 4)",248
timeStamp,str,19,[],68
attendance,int,0,[],28
venueName,str,12,[],61
referee,dict,5,"(officialId, firstName, lastName, hasParticipa...",248
weatherCode,str,1,[],58
elapsed,str,2,[],51
startTime,str,19,[],68
startDate,str,19,[],68


In [ ]:
def print_onsite_url(ws_url):
    ws_id = ws_url.split("-")[-1].split(".")[0]
    print(f"https://whoscored.com/Matches/{ws_id}/Live/")

In [ ]:
print_onsite_url(ws_url)

### they create [$34M+](https://www.worthofweb.com/website-value/whoscored.com/) value with this!